<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lesson 6.02 - Bootstrapping and Bagging

## Importing libraries


We'll need the following libaries for today's lecture:
- `pandas`
- `numpy`
- `DecisionTreeClassifier` from `sklearn`'s `tree` module
- `BaggingClassifier` from `sklearn`'s `ensemble` module
- `train_test_split` and `cross_val_score` from `sklearn`'s `model_selection` module
- `accuracy_score` from `sklearn`'s `metrics` module

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

## Load the Data

We'll be using the `Heart.csv` from the [ISLR Website](https://www-bcf.usc.edu/~gareth/ISL/). There's a copy in this repo under `./datasets/Heart.csv`.

In [2]:
# Read in the Heart .csv data.
df = pd.read_csv('../datasets/Heart.csv')

# Drop the `Unnamed: 0` column.
df.drop('Unnamed: 0', axis=1, inplace=True)

# Check the first few rows to make sure we dropped the column properly.
df.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


## Data cleaning: Drop rows with null values

In [3]:
# Check the shape of the data.
df.shape

(303, 14)

In [4]:
# How much missing data do we have?
df.isnull().sum()

Age          0
Sex          0
ChestPain    0
RestBP       0
Chol         0
Fbs          0
RestECG      0
MaxHR        0
ExAng        0
Oldpeak      0
Slope        0
Ca           4
Thal         2
AHD          0
dtype: int64

In [5]:
# Drop NAs.
df.dropna(inplace=True)

In [6]:
# Confirm all missing data is dropped.
df.isnull().sum()

Age          0
Sex          0
ChestPain    0
RestBP       0
Chol         0
Fbs          0
RestECG      0
MaxHR        0
ExAng        0
Oldpeak      0
Slope        0
Ca           0
Thal         0
AHD          0
dtype: int64

In [7]:
# What's the shape of our data now?
df.shape

(297, 14)

## Feature Engineering

In [7]:
# Create dummies for the `ChestPain`, `Thal`, and `AHD` columns.
# Be sure to set `drop_first=True`.
df = pd.get_dummies(df,
                    columns=['ChestPain', 'Thal', 'AHD'],
                    drop_first=True)

# Confirm we did this correctly.
df.head()

,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,ChestPain_nonanginal,ChestPain_nontypical,ChestPain_typical,Thal_normal,Thal_reversable,AHD_Yes
0,63,1,145,233,1,2,150,0,2.3,3,0.0,0,0,1,0,0,0
1,67,1,160,286,0,2,108,1,1.5,2,3.0,0,0,0,1,0,1
2,67,1,120,229,0,2,129,1,2.6,2,2.0,0,0,0,0,1,1
3,37,1,130,250,0,0,187,0,3.5,3,0.0,1,0,0,1,0,0
4,41,0,130,204,0,2,172,0,1.4,1,0.0,0,1,0,1,0,0


## Model Prep: Create `X` and `y` variables

Our target column will be `AHD_Yes`: 
- 1 means the patient has heart disease
- 0 means they aren't diagnosed with heart disease

In [8]:

X = df.drop('AHD_Yes', axis='columns')

y = df['AHD_Yes']

In [9]:
# What is the accuracy of our baseline model?
y.value_counts(normalize=True)

0    0.538721
1    0.461279
Name: AHD_Yes, dtype: float64

<details><summary>What does a false positive mean in this case?</summary>
    
- A false positive indicates someone **falsely** predict as being in the **positive** class.
- This is someone we incorrectly think has heart disease.
- Incorrectly predicting someone to have heart disease is bad... but it _might_ be worse to incorrectly predict that someone is healthy!
</details>

## Model Prep: Train/Test Split

In [21]:
# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    stratify=y)

## Model Instantiation

In [22]:
# Instantiate `DecisionTreeClassifier` object.
tree = DecisionTreeClassifier()

## Model Evaluation

In [23]:
# Get a cross_val_score for our tree.
cross_val_score(tree, X_train, y_train).mean()

/Users/gateam/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.7117117117117117

In [24]:
# Fit and score on the training data.
tree.fit(X_train, y_train)
tree.score(X_train, y_train)

1.0

In [25]:
# Score on the testing data.
tree.score(X_test, y_test)

0.6933333333333334

<details><summary> Where do decision trees tend to fall on the Bias/Variance spectrum?</summary>
    
- Decision trees very easily overfit.
- They tend to suffer from **high error due to variance**.
</details>

## Introduction to Ensemble Methods
We can list out the different types of models we've built thus far:
- Linear Regression
- Logistic Regression
- $k$-Nearest Neighbors
- Naive Bayes Classification _(maybe)_

If we want to use any of these models, we follow the same type of process.
1. Based on our problem, we identify which model to use. (Is our problem classification or regression? Do we want an interpretable model?)
2. Fit the model using the training data.
3. Use the fit model to generate predictions.
4. Evaluate our model's performance and, if necessary, return to step 2 and make changes.

So far, we've always had exactly one model. Today, however, we're going to talk about **ensemble methods**. Mentally, you should think about this as if we build multiple models and then aggregate their results in some way.

## Why would we build an "ensemble model?"

Our goal is to estimate $f$, the true function. (Think about $f$ as the **true process** that dictates Ames housing prices.)

We can come up with different hypotheses $h_1$, $h_2$, and so on to get as close to $f$ as possible. (Think about $h_1$ as the model you built to predict $f$, think of $h_2$ as the model your neighbor built to predict $f$, and so on.)

![](./assets/images/ensemble-benefits.png)


### (Advanced) Three Benefits: Statistical, Computational, Representational
- The **statistical** benefit to ensemble methods: By building one model, our predictions are almost certainly going to be wrong. Predictions from one model might overestimate housing prices; predictions from another model might underestimate housing prices. By "averaging" predictions from multiple models, we'll see that we can often cancel our errors out and get closer to the true function $f$.
- The **computational** benefit to ensemble methods: It might be impossible to develop one model that globally optimizes our objective function. (Remember that CART reach locally-optimal solutions that aren't guaranteed to be the globally-optimal solution.) In these cases, it may be **impossible** for one CART to arrive at the true function $f$. However, generating many different models and averaging their predictions may allow us to get results that are closer to the global optimum than any individual model.
- The **representational** benefit to ensemble methods: Even if we had all the data and all the computer power in the world, it might be impossible for one model to **exactly** equal $f$. For example, a linear regression model can never model a relationship where a one-unit change in $X$ is associated with some *different* change in $Y$ based on the value of $X$. All models have some shortcomings. (See [the no free lunch theorems](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization).) While individual models have shortcomings, by creating multiple models and aggregating their predictions, we can actually create predictions that represent something that one model cannot ever represent.

We can summarize this as the **wisdom of the crowd**.

## Wisdom of the Crowd: Guess the weight of Penelope

![](./assets/penelope.jpg)

[Image source: https://www.npr.org](https://www.npr.org/sections/money/2015/07/17/422881071/how-much-does-this-cow-weigh)

## Ensemble predictions

Let's mimic the "wisdom of the crowd" by creating several decision trees and averaging their predictions on the test set.

In [26]:
# What is this line doing?
predictions = pd.DataFrame(index=X_test.index)

In [27]:
# Generate ten decision trees.
for i in range(10):
    # Instantiate decision trees.
    t = DecisionTreeClassifier()
    # Fit to our training data.
    t.fit(X_train, y_train)
    
    # Put predictions in dataframe.
    predictions[f'Tree {i}'] = t.predict(X_test)

predictions

,Tree 0,Tree 1,Tree 2,Tree 3,Tree 4,Tree 5,Tree 6,Tree 7,Tree 8,Tree 9
113,1,1,1,1,1,1,1,1,1,1
195,1,1,1,1,1,1,1,1,1,1
64,1,1,1,1,1,1,1,1,1,1
27,0,0,0,0,0,0,0,0,0,0
245,0,0,0,0,0,0,0,0,0,0
210,0,0,0,0,0,0,0,0,0,0
221,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0
116,0,0,0,0,0,0,0,0,0,0
157,1,1,1,1,1,1,1,1,1,1


In [28]:
# Generate aggregated predicted probabilities.
probs = predictions.mean(axis='columns')

In [29]:
# Check out probs.
probs

113    1.0
195    1.0
64     1.0
27     0.0
245    0.0
210    0.0
221    1.0
1      0.0
116    0.0
157    1.0
194    0.0
190    0.0
298    1.0
103    0.0
278    0.0
5      0.0
102    1.0
289    0.0
187    1.0
21     1.0
90     1.0
66     0.0
104    0.2
162    0.0
297    0.0
232    0.0
56     1.0
252    1.0
155    1.0
34     0.0
      ... 
265    0.0
135    0.0
169    0.0
236    1.0
163    0.0
214    0.8
23     0.6
172    0.4
272    1.0
300    0.0
275    0.5
255    0.0
198    0.0
242    0.0
270    1.0
280    1.0
137    1.0
281    0.0
3      1.0
132    0.0
183    1.0
62     1.0
168    0.4
70     0.0
295    0.0
93     1.0
133    0.0
33     0.0
20     0.6
76     1.0
Length: 75, dtype: float64

In [30]:
accuracy_score(y_test, (probs > .5).astype(int))

0.6933333333333334

## Ensemble evaluation?

<details><summary>Why didn't our score improve?</summary>

- Because we fit the same model on the same data ten times!
</details>

## Bootstrapping

Let's get started actually making ensemble predictions. However, in order to do that, we'll need to introduce the idea of bootstrapping, or **random sampling with replacement.**

### Summary
When bootstrapping in order to fit multiple estimators, we want to:
- Take a sample of size $n$.
- With replacement.
- From our original data.

<details><summary>Why do you think we want to take a sample of size n?</summary>
    
- Because we want our estimators to be fit on data of the same size!
- If our original data had $n = 1,000$ and we bootstrapped a sample of size 50 to fit an estimator on, our estimator fit on size 50 will probably look very, very different from an estimator fit on size 1,000.
</details>

<details><summary>Why do you think we want to sample with replacement?</summary>
    
- If we didn't sample with replacement, we'd just get identical samples of size $n$. (These would be copies of our original data!)
</details>

Boostrapping with `pandas`:

In [31]:
X_train.sample(n = X_train.shape[0],
               replace = True,
               random_state = 42)

,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,ChestPain_nonanginal,ChestPain_nontypical,ChestPain_typical,Thal_normal,Thal_reversable
59,51,1,125,213,0,2,125,1,1.4,1,1.0,0,0,1,1,0
234,54,0,160,201,0,0,163,0,0.0,1,1.0,1,0,0,1,0
50,41,0,105,198,0,0,168,0,0.0,1,1.0,0,1,0,1,0
117,35,0,138,183,0,0,182,0,1.4,1,0.0,0,0,0,1,0
10,57,1,140,192,0,0,148,0,0.4,2,0.0,0,0,0,0,0
233,74,0,120,269,0,2,121,1,0.2,1,1.0,0,1,0,1,0
291,55,0,132,342,0,0,166,0,1.2,1,0.0,0,1,0,1,0
158,60,1,140,293,0,2,170,0,1.2,2,2.0,0,0,0,0,1
59,51,1,125,213,0,2,125,1,1.4,1,1.0,0,0,1,1,0
136,70,1,145,174,0,0,125,1,2.6,3,0.0,0,0,0,0,1


## Bagging: Bootstrap Aggregating

As we have seen, decision trees are very powerful machine learning models. However, decision trees have some limitations. In particular, trees that are grown very deep tend to learn highly irregular patterns (a.k.a. they overfit their training sets). 

Bagging (bootstrap aggregating) mitigates this problem by exposing different trees to different sub-samples of the training set.

The process for creating bagged decision trees is as follows:
1. From the original data of size $n$, bootstrap $k$ samples each of size $n$ (with replacement!).
2. Build a decision tree on each bootstrapped sample.
3. Make predictions by passing a test observation through all $k$ trees and developing one aggregate prediction for that observation.

![](./assets/Ensemble.png)

### What do you mean by "aggregate prediction?"
As with all of our modeling techniques, we want to make sure that we can come up with one final prediction for our observation. (Building 1,000 trees and coming up with 1,000 predictions for one observation probably wouldn't be very helpful.)

Suppose we want to predict whether or not a Reddit post is going to go viral, where `1` indicates viral and `0` indicates non-viral. We build 100 decision trees. Given a new Reddit post labeled `X_test`, we pass these features into all 100 decision trees.
- 70 of the trees predict that the post in `X_test` will go viral.
- 30 of the trees predict that the post in `X_test` will not go viral.

**`.predict_proba(X_test)` to do?**
<details><summary>What might you expect .predict(X_test) do?</summary>

- `.predict(X_test)` should output a 1, predicting that the post will go viral.

</details>

<details><summary>What might you expect .predict_proba(X_test) do?</summary>

- `.predict_proba(X_test)` should output 0.7, indicating the probability of the post going viral is 70%.
</details>


## Bagging Classifier using a `for` loop

In the cell below, we'll create an ensemble of trees like before, except this time we'll train each tree to a **bootstrapped** sample of the training data.

In [37]:
# Instantiate dataframe.
predictions = pd.DataFrame(index=X_test.index)

# Generate ten decision trees.
for i in range(10):
    
    # Bootstrap X data.
    # Should we add a random seed?
    X_sample = X_train.sample(n = X_train.shape[0],
                              replace=True)
    
    # Get y data that matches the X data.
    y_sample = y_train[X_sample.index]
    
    # Instantiate decision trees.
    t = DecisionTreeClassifier()
    
    # Fit to our training data.
    t.fit(X_train, y_train)
    
    # Put predictions in dataframe.
    predictions[f'Tree {i}'] = t.predict(X_test)

predictions.head(20)

,Tree 0,Tree 1,Tree 2,Tree 3,Tree 4,Tree 5,Tree 6,Tree 7,Tree 8,Tree 9
113,1,1,1,1,1,1,1,1,1,1
195,1,1,1,1,1,1,1,1,1,1
64,1,1,1,1,1,1,1,1,1,1
27,0,0,0,0,0,0,0,0,0,0
245,0,0,0,0,0,0,0,0,0,0
210,0,0,0,0,0,0,0,0,0,0
221,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0
116,0,0,0,0,0,0,0,0,0,0
157,1,1,1,1,1,1,1,1,1,1


In [38]:
# Generate aggregated predicted probabilities.
predictions.mean(axis='columns')

113    1.0
195    1.0
64     1.0
27     0.0
245    0.0
210    0.0
221    1.0
1      0.0
116    0.0
157    1.0
194    0.0
190    0.0
298    1.0
103    0.0
278    0.0
5      0.0
102    1.0
289    0.0
187    1.0
21     1.0
90     1.0
66     0.0
104    0.2
162    0.0
297    0.0
232    0.0
56     1.0
252    1.0
155    1.0
34     0.0
      ... 
265    0.0
135    0.0
169    0.0
236    1.0
163    0.0
214    1.0
23     0.4
172    0.7
272    1.0
300    0.0
275    0.3
255    0.0
198    0.0
242    0.0
270    1.0
280    1.0
137    1.0
281    0.0
3      1.0
132    0.0
183    1.0
62     1.0
168    0.7
70     0.0
295    0.0
93     1.0
133    0.0
33     0.0
20     0.3
76     1.0
Length: 75, dtype: float64

In [39]:
# Generate aggregated predicted probabilities as probs.
probs = predictions.mean(axis='columns')

In [40]:
accuracy_score(y_test, (probs >= .5).astype(int))

0.72

## Bagging Classifier using `sklearn`

[BaggingClassifier Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)

In the cell below, create and score instance of `BaggingClassifier` on the test set. You should get a similar score to the one in the previous step.

In [44]:
# Instantiate BaggingClassifier.
bag = BaggingClassifier(random_state = 42)

# Fit BaggingClassifier.
bag.fit(X_train, y_train)

# Score BaggingClassifier.
bag.score(X_test, y_test)

0.7733333333333333